# LUMIN.AI: Sentiment Analysis MVP

This notebook implements a minimum viable product for sentiment analysis in the LUMIN.AI project.

In [ ]:
# Import libraries
from transformers import pipeline


# Set up sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")